In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Users')

+---------+-----------+-------------------------+
| user_id |    name   |           mail          |
+---------+-----------+-------------------------+
|    1    |  Winston  |   winston@leetcode.com  |
|    2    |  Jonathan |     jonathanisgreat     |
|    3    | Annabelle |   bella-@leetcode.com   |
|    4    |   Sally   | sally.come@leetcode.com |
|    5    |   Marwan  | quarz#2020@leetcode.com |
|    6    |   David   |    david69@gmail.com    |
|    7    |  Shapiro  |   .shapo@leetcode.com   |
+---------+-----------+-------------------------+


### Find the users who have valid emails.

#### *A valid e-mail has a prefix name and a domain where:*

* The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.', and/or dash '-'. 
* The prefix name must start with a letter.

#### *The domain is '@leetcode.com'.*
```
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
+---------+-----------+-------------------------+
Explanation: 
The mail of user 2 does not have a domain.
The mail of user 5 has the # sign which is not allowed.
The mail of user 6 does not have the leetcode domain.
The mail of user 7 starts with a period.
```

# Using `regex`

In [3]:
%%sql 

SELECT
    mail
FROM Users
WHERE mail REGEXP '^[a-zA-Z]'


mail
winston@leetcode.com
jonathanisgreat
bella-@leetcode.com
sally.come@leetcode.com
quarz#2020@leetcode.com
david69@gmail.com


In [4]:
%%sql 

-- Need to wrap '.' character to pass 'leetcode?com' test case 

SELECT
    user_id,
    name,
    mail
FROM Users
WHERE mail REGEXP '^[a-zA-Z][a-zA-Z0-9\.\_\-]*@leetcode[.]com$'


user_id,name,mail
1,Winston,winston@leetcode.com
3,Annabelle,bella-@leetcode.com
4,Sally,sally.come@leetcode.com


In [5]:
%%sql 

SELECT 
    mail
FROM Users
WHERE mail REGEXP '^[a-zA-Z][a-zA-Z0-9\.\_\-]*@leetcode\.com$'

mail
winston@leetcode.com
bella-@leetcode.com
sally.come@leetcode.com


## Using `SUBSTRING_INDEX`

In [6]:
%%sql

SELECT 
    mail
FROM Users
WHERE mail LIKE "%@leetcode.com"

mail
winston@leetcode.com
bella-@leetcode.com
sally.come@leetcode.com
quarz#2020@leetcode.com
.shapo@leetcode.com


In [7]:
%%sql

SELECT 
    SUBSTRING_INDEX(mail, '@', 1)
FROM Users
WHERE mail LIKE "%@leetcode.com"

"SUBSTRING_INDEX(mail, '@', 1)"
winston
bella-
sally.come
quarz#2020
.shapo


In [8]:
%%sql

SELECT 
    user_id,
    name,
    mail
FROM Users
WHERE mail LIKE "%@leetcode.com" 
    AND SUBSTRING_INDEX(mail, '@', 1) NOT LIKE '%#%'
    AND SUBSTRING_INDEX(mail, '@', 1) NOT LIKE '.%'

user_id,name,mail
1,Winston,winston@leetcode.com
3,Annabelle,bella-@leetcode.com
4,Sally,sally.come@leetcode.com


# Using Pandas

In [9]:
users_query = %sql SELECT * FROM Users # type: ignore
users_df = users_query.DataFrame()

display(users_df)

,user_id,name,mail
0,1,Winston,winston@leetcode.com
1,2,Jonathan,jonathanisgreat
2,3,Annabelle,bella-@leetcode.com
3,4,Sally,sally.come@leetcode.com
4,5,Marwan,quarz#2020@leetcode.com
5,6,David,david69@gmail.com
6,7,Shapiro,.shapo@leetcode.com


In [10]:
filtered_df = users_df[users_df['mail'].str.contains('@leetcode.com')]
filtered_df

,user_id,name,mail
0,1,Winston,winston@leetcode.com
2,3,Annabelle,bella-@leetcode.com
3,4,Sally,sally.come@leetcode.com
4,5,Marwan,quarz#2020@leetcode.com
6,7,Shapiro,.shapo@leetcode.com


In [11]:
filtered_df['mail'].str.split('@')

0       [winston, leetcode.com]
2        [bella-, leetcode.com]
3    [sally.come, leetcode.com]
4    [quarz#2020, leetcode.com]
6        [.shapo, leetcode.com]
Name: mail, dtype: object

In [12]:
filtered_df['mail'].str.split('@')

0       [winston, leetcode.com]
2        [bella-, leetcode.com]
3    [sally.come, leetcode.com]
4    [quarz#2020, leetcode.com]
6        [.shapo, leetcode.com]
Name: mail, dtype: object

In [13]:
filtered_df['mail'].str.split('@').str.get(0)

0       winston
2        bella-
3    sally.come
4    quarz#2020
6        .shapo
Name: mail, dtype: object

In [14]:
filtered_df['email_local_part'] = filtered_df['mail'].str.split('@').str.get(0)

filtered_df

/var/folders/2w/g1xqblv11xdc789plh7y3tsw0000gn/T/ipykernel_37990/2339840383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['email_local_part'] = filtered_df['mail'].str.split('@').str.get(0)


,user_id,name,mail,email_local_part
0,1,Winston,winston@leetcode.com,winston
2,3,Annabelle,bella-@leetcode.com,bella-
3,4,Sally,sally.come@leetcode.com,sally.come
4,5,Marwan,quarz#2020@leetcode.com,quarz#2020
6,7,Shapiro,.shapo@leetcode.com,.shapo


In [15]:
# Filter out those that start with a letter
pattern = r'^[a-zA-Z]'
filtered_df['email_local_part'].str.contains(pattern, regex=True)

0     True
2     True
3     True
4     True
6    False
Name: email_local_part, dtype: bool

In [16]:
#  Matches zero or more occurrences of any uppercase or lowercase letter, digit, underscore, period, or dash.
pattern = r'^[a-zA-Z][a-zA-Z0-9_.-]*'
filtered_df['email_local_part'].str.contains(pattern, regex=True)

0     True
2     True
3     True
4     True
6    False
Name: email_local_part, dtype: bool

### Without the `$` end-of-string(EOS) anchor, the pattern only requires the local part to start with a letter and followed by zero or more alphanumeric characters, underscores, periods, or dashes. It doesn't account for any additional characters after the valid local part, so the regex matches only the art that satisfies the pattern and ignores the rest of the email.

In [17]:
# Ensure that the pattern matches the entire string from start to end, leaving no room for additional
# characters like '#' after the valid part.
pattern = r'^[a-zA-Z][a-zA-Z0-9_.-]*$'
filtered_df['email_local_part'].str.contains(pattern, regex=True)

0     True
2     True
3     True
4    False
6    False
Name: email_local_part, dtype: bool

In [18]:
mask = filtered_df['email_local_part'].str.contains(pattern, regex=True)
final_df = filtered_df[mask]
final_df

,user_id,name,mail,email_local_part
0,1,Winston,winston@leetcode.com,winston
2,3,Annabelle,bella-@leetcode.com,bella-
3,4,Sally,sally.come@leetcode.com,sally.come


In [19]:
final_df.drop('email_local_part', axis=1, inplace=True)
final_df

/var/folders/2w/g1xqblv11xdc789plh7y3tsw0000gn/T/ipykernel_37990/473826979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop('email_local_part', axis=1, inplace=True)


,user_id,name,mail
0,1,Winston,winston@leetcode.com
2,3,Annabelle,bella-@leetcode.com
3,4,Sally,sally.come@leetcode.com
